In [1]:
import torch
from torch import nn, optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Cargamos el dataset Iris
X, y = load_iris(return_X_y=True)

# Normalizamos los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Lo convertimos a tensores de PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Dividimos en conjunto de entrenamiento y de test
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Imprimimos los 10 primeros ejemplos y sus etiquetas
print(X_train[:10])
print(y_train[:10])
print("Longitud del conjunto de entrenamiento:", len(X_train))
print("Longitud del conjunto de test:", len(X_test))

tensor([[-1.5065,  1.2492, -1.5676, -1.3154],
        [-0.1737,  3.0908, -1.2834, -1.0522],
        [ 1.0380,  0.0982,  0.3649,  0.2641],
        [-1.2642,  0.7888, -1.2266, -1.3154],
        [-1.7489,  0.3284, -1.3971, -1.3154],
        [ 0.5533, -1.2830,  0.7059,  0.9223],
        [ 0.6745,  0.3284,  0.4217,  0.3958],
        [-0.7795,  1.0190, -1.2834, -1.3154],
        [-1.0218,  1.2492, -1.3402, -1.3154],
        [-0.7795,  2.4002, -1.2834, -1.4471]])
tensor([0, 0, 1, 0, 0, 2, 1, 0, 0, 0])
Longitud del conjunto de entrenamiento: 120
Longitud del conjunto de test: 30


In [2]:
# Definimos el modelo
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 5) # 4 características de entrada, 5 neuronas en la capa oculta
        self.fc2 = nn.Linear(5, 3) # 3 clases de salida

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # Fíjate en que no aplicamos la función de activación softmax, ya que la función de pérdida CrossEntropyLoss lo hace por nosotros

model = IrisNet()

print(model)
print("Número de parámetros:", sum(p.numel() for p in model.parameters()))

IrisNet(
  (fc1): Linear(in_features=4, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=3, bias=True)
)
Número de parámetros: 43


In [3]:
# Definimos el criterio de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [4]:
# Entrenamos el modelo
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X_train)  # Pasamos el conjunto de entrenamiento completo, rara vez se hace esto en la práctica
    loss = criterion(output, y_train)  # Date cuenta de que pasamos las etiquetas originales, no en formato one-hot
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 1.1032124757766724
Epoch 11, Loss: 0.9077782034873962
Epoch 21, Loss: 0.6948903799057007
Epoch 31, Loss: 0.5140151381492615
Epoch 41, Loss: 0.39988014101982117
Epoch 51, Loss: 0.3256687819957733
Epoch 61, Loss: 0.2662292420864105
Epoch 71, Loss: 0.21919700503349304
Epoch 81, Loss: 0.18273359537124634
Epoch 91, Loss: 0.1535124033689499


In [5]:
# Evaluamos el modelo
with torch.no_grad():
    correct = 0
    total = y_test.size(0)
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == y_test).sum().item()

print(f'Accuracy of the model on the test set: {round(100 * correct / total, 2)}%')

Accuracy of the model on the test set: 100.0%
